In [114]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

TensorFlow is already loaded. Please restart the runtime to change versions.
2.4.1


In [115]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [116]:
!ls data

 adolescent  'new born'  'old age'   output  'young adult'


In [117]:
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)

In [119]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))
all_images[:5]

['data/adolescent/adolescent_0.png',
 'data/adolescent/adolescent_1.png',
 'data/adolescent/adolescent_10.png',
 'data/adolescent/adolescent_100.png',
 'data/adolescent/adolescent_101.png']

In [120]:
random.seed(42)
random.shuffle(all_images)
all_images[:5]


['data/adolescent/adolescent_175.png',
 'data/young adult/youngadult_63.png',
 'data/new born/newborn_105.png',
 'data/old age/oldage_247.png',
 'data/young adult/youngadult_65.png']

In [121]:
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [122]:
data[0]


array([[[188., 189., 193.],
        [201., 200., 202.],
        [208., 208., 208.],
        ...,
        [248., 248., 248.],
        [248., 248., 248.],
        [247., 247., 247.]],

       [[189., 190., 194.],
        [201., 200., 201.],
        [211., 210., 210.],
        ...,
        [249., 249., 249.],
        [249., 249., 249.],
        [249., 249., 249.]],

       [[189., 190., 194.],
        [202., 202., 202.],
        [213., 211., 211.],
        ...,
        [250., 250., 250.],
        [249., 249., 249.],
        [249., 249., 249.]],

       ...,

       [[145., 147., 148.],
        [152., 154., 155.],
        [153., 154., 156.],
        ...,
        [247., 247., 247.],
        [247., 247., 247.],
        [247., 247., 247.]],

       [[143., 146., 147.],
        [153., 155., 156.],
        [156., 158., 159.],
        ...,
        [246., 246., 246.],
        [247., 247., 247.],
        [246., 246., 246.]],

       [[158., 163., 166.],
        [170., 173., 177.],
        [184., 1

In [123]:
#data = np.array(data, dtype='float' )/255.
#labels = np.array(labels)
#le = LabelEncoder()
#labels1 = le.fit_transform(labels)

In [131]:
# LABELS 
# 0-1-> Cat and DOG-> [0 1] , [1 0]
data = np.array(data, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0]])

In [132]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels, test_size=0.25, random_state=42)

In [133]:
#Augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.2, 
                         height_shift_range=0.2, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [134]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.20
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 
HP_img_dims = (48,48,3)

In [135]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth,classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))


    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))

    return model

In [150]:
from tensorflow.keras.optimizers import Adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1], depth=HP_img_dims[2],classes = len(lb.classes_))
opt = Adam(lr=HP_init_lr, decay = HP_init_lr/ HP_epoch)
model.compile(loss='categorical_crossentropy', optimizer= 'adam', metrics=['accuracy'])

In [151]:
len(trainx)


755

In [152]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


In [153]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [154]:
cnn1_history = model.fit(aug.flow(trainx, trainy),
                        batch_size= 512,
                        validation_data= (testx, testy),
                        epochs=30,
                        steps_per_epoch = len(trainx)// HP_batch_size,
                        callbacks=[early_stop],
                        shuffle=False    # shuffle=False to reduce randomness and increase reproducibility
                       )

Epoch 1/30


InvalidArgumentError: ignored